# VacationPy
----

#### Note
* Keep an eye on API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor usage and billing.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint


# Import API key
from config import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import the clean_city_data from CSV
clean_city_data = pd.read_csv('../Resources/clean_city_data.csv', encoding='utf-8')
del clean_city_data['Unnamed: 0']
clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Dikson,73.51,80.55,2.16,81.0,100.0,5.47,RU,1.601447e+09
1,Gandia,38.97,-0.18,16.11,8.0,56.0,0.45,ES,1.601447e+09
2,Lautaro,-38.52,-72.45,7.00,93.0,75.0,4.10,CL,1.601447e+09
3,Iskateley,67.68,53.15,8.49,82.0,10.0,4.99,RU,1.601447e+09
4,Murray Bridge,-35.12,139.27,14.44,93.0,90.0,4.47,AU,1.601447e+09
...,...,...,...,...,...,...,...,...,...
515,Hilo,19.73,-155.09,27.00,74.0,20.0,2.10,US,1.601447e+09
516,Atuona,-9.80,-139.03,25.96,75.0,2.0,9.94,PF,1.601447e+09
517,Ozernovskiy,51.50,156.52,10.51,81.0,34.0,1.85,RU,1.601447e+09
518,Yellowknife,62.46,-114.35,3.33,100.0,20.0,3.10,CA,1.601447e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_city_data[["Lat", "Lng"]].astype(float)
# locations
humidity = clean_city_data["Humidity"].astype(float)
# humidity

# Create Humidity Heatmap layer
fig = gmaps.figure(zoom_level=2, center=[0,0])

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clean_city_data.head(1)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Dikson,73.51,80.55,2.16,81.0,100.0,5.47,RU,1.601447e+09


In [6]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 30 degrees but higher than 20 Celsius.
# # Wind speed less than 10 mph.
# # Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned 
# by your API requests to a reasonable number.

In [7]:
ideal_weather = clean_city_data.loc[(
    (clean_city_data['Max Temp'] > 20) & (clean_city_data['Max Temp'] < 30) 
    & (clean_city_data['Wind Speed'] < 10) & (clean_city_data['Cloudiness'] == 0))]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Nouadhibou,20.93,-17.03,21.00,77.0,0.0,7.20,MR,1.601447e+09
41,Muhafazat Lahij,13.17,44.58,28.84,57.0,0.0,3.04,YE,1.601447e+09
44,San Quintín,30.48,-115.95,24.53,55.0,0.0,2.76,MX,1.601447e+09
57,Geraldton,-28.77,114.60,23.00,49.0,0.0,5.70,AU,1.601447e+09
85,Yulara,-25.24,130.99,29.00,23.0,0.0,3.10,AU,1.601447e+09
93,Kiryat Gat,31.61,34.76,26.67,89.0,0.0,2.29,IL,1.601447e+09
100,Maţāy,28.42,30.78,28.81,46.0,0.0,6.42,EG,1.601447e+09
118,Adrar,20.50,-10.07,29.63,29.0,0.0,4.58,MR,1.601447e+09
179,Abu Samrah,35.30,37.18,28.50,39.0,0.0,5.77,SY,1.601447e+09
193,Salani,29.96,79.71,28.15,37.0,0.0,2.07,IN,1.601447e+09


In [8]:
len(ideal_weather)

14

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store ideal_weather cities into variable named hotel_df
hotel_df = pd.DataFrame(ideal_weather)
hotel_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nouadhibou,20.93,-17.03,21.00,77.0,0.0,7.20,MR,1.601447e+09
1,Muhafazat Lahij,13.17,44.58,28.84,57.0,0.0,3.04,YE,1.601447e+09
2,San Quintín,30.48,-115.95,24.53,55.0,0.0,2.76,MX,1.601447e+09
3,Geraldton,-28.77,114.60,23.00,49.0,0.0,5.70,AU,1.601447e+09
4,Yulara,-25.24,130.99,29.00,23.0,0.0,3.10,AU,1.601447e+09
5,Kiryat Gat,31.61,34.76,26.67,89.0,0.0,2.29,IL,1.601447e+09
6,Maţāy,28.42,30.78,28.81,46.0,0.0,6.42,EG,1.601447e+09
7,Adrar,20.50,-10.07,29.63,29.0,0.0,4.58,MR,1.601447e+09
8,Abu Samrah,35.30,37.18,28.50,39.0,0.0,5.77,SY,1.601447e+09
9,Salani,29.96,79.71,28.15,37.0,0.0,2.07,IN,1.601447e+09


In [10]:
# Set parameters to search for hotels within 5000 meters.
# geocoordinates
target_coordinates = "34.01,-118.81"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()
pprint.pprint(response)


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 34.0166744, 'lng': -118.815994},
                           'viewport': {'northeast': {'lat': 34.01811422989272,
                                                      'lng': -118.8149439701073},
                                        'southwest': {'lat': 34.01541457010727,
                                                      'lng': -118.8176436298928}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'name': 'Malibu Country Inn',
              'photos': [{'height': 3464,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/103690878833767175900">leslie '
                                                'telles</a>'],
                          'photo_reference': 'CmRaAAAAk4OnDQecAawZ8X8isiZdqSf0PqWnANgoiWfhg_M-jIJwu1JcD

In [11]:
response['results'][0]['name']

'Malibu Country Inn'

In [12]:
# Hit the Google Places API for each city's coordinates.

# Variable to store first returned result
hotels = []

print("Beginning search for nearby hotels... \n-----------------------------")

# Loop through the hotel_df and run a lat/long search for each city (05-Stu_Google_Complex/Solved/Airport_Ratings)
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Update params coords variable
    params["location"] = f"{lat},{lng}"
#     print(params['location'])
    try:
        print(f"Searching hotels near {row['City']}")
        hotel_search = requests.get(base_url, params=params).json()
#         print('response')
        hotels.append(hotel_search['results'][0]['name'])
        print('--->hotel appended')
        
    except KeyError and IndexError:
        print(f"X--X No hotels found near enough to this location. Skipping {row['City']}...")
        hotels.append(None)
        


Beginning search for nearby hotels... 
-----------------------------
Searching hotels near Nouadhibou
--->hotel appended
Searching hotels near Muhafazat Lahij
X--X No hotels found near enough to this location. Skipping Muhafazat Lahij...
Searching hotels near San Quintín
--->hotel appended
Searching hotels near Geraldton
--->hotel appended
Searching hotels near Yulara
--->hotel appended
Searching hotels near Kiryat Gat
X--X No hotels found near enough to this location. Skipping Kiryat Gat...
Searching hotels near Maţāy
--->hotel appended
Searching hotels near Adrar
X--X No hotels found near enough to this location. Skipping Adrar...
Searching hotels near Abu Samrah
X--X No hotels found near enough to this location. Skipping Abu Samrah...
Searching hotels near Salani
X--X No hotels found near enough to this location. Skipping Salani...
Searching hotels near Ridgecrest
--->hotel appended
Searching hotels near Cabo San Lucas
--->hotel appended
Searching hotels near Aracati
X--X No hotels 

In [13]:
hotels

['Free Zone Hotel Nouadhibou',
 None,
 'Old Mill Hotel',
 'The Gerald Apartment Hotel',
 'Sails in the Desert',
 None,
 'Nag Aespen',
 None,
 None,
 None,
 'SpringHill Suites by Marriott Ridgecrest',
 'Welk Resorts Cabo San Lucas - Sirena del Mar',
 None,
 None]

In [14]:
# Store the first Hotel result into the DataFrame.
hotel_df['Hotel Name'] = hotels
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Nouadhibou,20.93,-17.03,21.00,77.0,0.0,7.20,MR,1.601447e+09,Free Zone Hotel Nouadhibou
44,San Quintín,30.48,-115.95,24.53,55.0,0.0,2.76,MX,1.601447e+09,Old Mill Hotel
57,Geraldton,-28.77,114.60,23.00,49.0,0.0,5.70,AU,1.601447e+09,The Gerald Apartment Hotel
85,Yulara,-25.24,130.99,29.00,23.0,0.0,3.10,AU,1.601447e+09,Sails in the Desert
100,Maţāy,28.42,30.78,28.81,46.0,0.0,6.42,EG,1.601447e+09,Nag Aespen
215,Ridgecrest,35.62,-117.67,22.22,19.0,0.0,0.79,US,1.601447e+09,SpringHill Suites by Marriott Ridgecrest
241,Cabo San Lucas,22.89,-109.91,27.78,69.0,0.0,0.67,MX,1.601447e+09,Welk Resorts Cabo San Lucas - Sirena del Mar


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]] 


In [17]:
# Add marker layer ontop of heat map
fig = gmaps.figure(zoom_level=2, center=[0,0])

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info, display_info_box=None)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))